In [1]:
import fastf1 as ff1
import pandas as pd
import time
from DB.models import Circuit, RacingWeekend, Driver, Session, SessionResult, init_db


In [2]:
time_cache = time.time()
sessionData = ff1.get_session(2024, 1, "Race")
sessionData.load()
time_cache = time.time() - time_cache


req         WARNING 	DEFAULT CACHE ENABLED! (101.32 MB) /root/.cache/fastf1
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10

In [3]:
ff1.Cache.set_disabled()
time_no_cache = time.time()
sessionData = ff1.get_session(2024, 1, "Race")
sessionData.load()
time_no_cache = time.time() - time_no_cache


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.0]
_api           INFO 	Fetching session info data...
_api           INFO 	Fetching driver list...
_api           INFO 	Fetching session status data...
_api           INFO 	Fetching lap count data...
_api           INFO 	Fetching track status data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
_api           INFO 	Fetching timing app data...
core           INFO 	Processing timing data...
_api           INFO 	Fetching car data...
_api           INFO 	Parsing car data...
_api           INFO 	Fetching position data...
_api           INFO 	Parsing position data...
_api           INFO 	Fetching weather data...
_api           INFO 	Fetching race control messages...
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']


In [4]:
print(f"Time taken with caching: {time_cache:.2f} seconds")
print(f"Time taken without caching: {time_no_cache:.2f} seconds")

Time taken with caching: 4.95 seconds
Time taken without caching: 26.30 seconds


In [5]:
engine, db_session = init_db()

In [6]:
time_db = time.time()

session = (
	db_session.query(Session)
	.join(RacingWeekend, Session.weekend_id == RacingWeekend.racing_weekend_id)
	.filter(
		RacingWeekend.year == 2024,
		RacingWeekend.round == 1,
		Session.session_type == "Race"
	)
	.first()
)

laps_data = []
for lap in session.laps:

	# add a row for each sector
	laps_data.append({
		"lap_num": lap.lap_num,
		"lap_time": lap.lap_time,
		"stint_num": lap.stint_num,
		"stint_lap": lap.stint_lap,
		"position": lap.position,
		"driver_name": lap.driver.driver_name,
		"driver_number": lap.driver.driver_num,
		"lap_time": lap.lap_time,
		
		"sector1_time": lap.s1_time,
		"sector2_time": lap.s2_time,
		"sector3_time": lap.s3_time,

		"tyre_type": lap.tyre_type,
		"tyre_laps": lap.tyre_laps,
		"pit": lap.pit,
		"pit_time": lap.pit_stop[0].pit_time if lap.pit_stop else None,
		"track_status": lap.track_status,
	})

race_df = pd.DataFrame(laps_data)

time_db = time.time() - time_db

In [7]:
print(f"Time taken with SQL DB: {time_db:.2f} seconds")

Time taken with SQL DB: 0.59 seconds
